In [1]:
from pprint import pprint
import pymysql
import pandas as pd

# 순서

1. SQL 에서 데이터 가져오고 DF로 만들기

2. 리뷰 문장구분 & feature 매칭

3. KW 사전 구축하기

4. KW로 SS / CS 구분하기

5. MPQA 사전으로 POS, NEG 라벨링

계속 순서대로 돌면서 코드 수정 & 보완할 것

앞으로 해야 할 것: CS에서 PRODUCT NAME 포함한 CS만 남기기

## 1. SQL 에서 데이터 가져오고 DF로 만들기

In [2]:

db = pymysql.connect(
    host='ec2-13-209-223-88.ap-northeast-2.compute.amazonaws.com',  # DATABASE_HOST
    port=3306,
    user='nlpuser',  # DATABASE_USERNAME
    passwd='1234',  # DATABASE_PASSWORD
    db='nlp',  # DATABASE_NAME
    charset='utf8'
)

SQL = "SELECT review FROM Reviews where Review like '%camera%' LIMIT 1000"
data = pd.read_sql(SQL, db)
data

,review
0,While many beginner DVDs try to teach you ever...
1,While many beginner DVDs try to teach you ever...
2,"I like to keep my screens clean, from my lapto..."
3,I bought this for my husband for his birthday ...
4,"I bought 2 of these, one for mom and one for m..."
5,I have owned the Nook Tablet since it was rele...
6,The Nook was disappointing for me. What reall...
7,"Months later, I am still happy with this Table..."
8,I got my Nook Tablet about two weeks ago. I co...
9,I went to a few places just to try testing the...


## 2. 리뷰 문장구분 & feature 매칭

In [3]:
import nltk
nltk.download('punkt')
from nltk.tokenize import sent_tokenize
from nltk.tokenize import word_tokenize

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\seonmin\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


#### 앞으로 추가해야함

In [4]:
feature={
    '12': ['resolution', 'pixel', 'megapixel'],   # 0
    '13': ['lens','wide angle','normal range'],   # 1
    '14': ['optical','zoom','optical zoom','digital zoom'],  # 2
    '15': ['memory','megabytes','MB'],
    '16': ['burst','continuous','shutter','recovery','motion','sport'],
    '17': ['battery','batteries','power'],
    '18': ['focus','exposure','manual','iso'],
    '19': ['LCD','screen'],
    '20': ['compression','compress','jpeg'],
    '21': ['flash','light']   # 9
}

#### feature 동의어 앞으로 추가해야함

In [5]:
db = pymysql.connect(
    host='ec2-13-209-223-88.ap-northeast-2.compute.amazonaws.com',  # DATABASE_HOST
    port=3306,
    user='nlpuser',  # DATABASE_USERNAME
    passwd='1234',  # DATABASE_PASSWORD
    db='nlp',  # DATABASE_NAME
    charset='utf8'
)

SQL = "SELECT * FROM Features"
feature_db = pd.read_sql(SQL, db)
feature_db

,FeatureID,CategoryID,TeamID,FeatureName,FeatureNameRe,UserIP,LastUpdatedDate
0,12,2,0,resolution,(resolution)|(pixel)|(megapixel),None,None
1,13,2,0,lens,(lens)|(wide angle)|(normal range),None,None
2,14,2,0,optical,(optical)|(zoom)|(optical zoom)|(digital zoom),None,None
3,15,2,0,memory,(memory)|(megabytes)|(MB),None,None
4,16,2,0,burst,(burst)|(continuous)|(shutter)|(recovery)|(mot...,None,None
5,17,2,0,battery,(battery)|(batteries)|(power),None,None
6,18,2,0,focus,(focus)|(exposure)|(manual)|(iso),None,None
7,19,2,0,LCD,(LCD)|(screen),None,None
8,20,2,0,compression,(compression)|(compress)|(jpeg),None,None
9,21,2,0,flash,(flash)|(light),None,None


In [6]:
feature_db = feature_db[['FeatureID', 'FeatureName', 'FeatureNameRe']]

In [7]:
feature_db

,FeatureID,FeatureName,FeatureNameRe
0,12,resolution,(resolution)|(pixel)|(megapixel)
1,13,lens,(lens)|(wide angle)|(normal range)
2,14,optical,(optical)|(zoom)|(optical zoom)|(digital zoom)
3,15,memory,(memory)|(megabytes)|(MB)
4,16,burst,(burst)|(continuous)|(shutter)|(recovery)|(mot...
5,17,battery,(battery)|(batteries)|(power)
6,18,focus,(focus)|(exposure)|(manual)|(iso)
7,19,LCD,(LCD)|(screen)
8,20,compression,(compression)|(compress)|(jpeg)
9,21,flash,(flash)|(light)


In [8]:
FeatureNameRe={
    '1': '(resolution)|(pixel)|(megapixel)',
    '2': '(lens)|(wide angle)|(normal range)',
    '3': '(optical)|(zoom)|(optical zoom)|(digital zoom)',
    '4': '(memory)|(megabytes)|(MB)',
    '5': '(burst)|(continuous)|(shutter)|(recovery)|(motion)|(sport)',
    '6': '(battery)|(batteries)|(power)',
    '7': '(focus)|(exposure)|(manual)|(iso)|(ISO)',
    '8': '(LCD)|(screen)',
    '9': '(compression)|(compress)|(jpeg)',
    '10': '(flash)|(light)'
}

### for문으로 문장마다 feature label 붙이기

for문 four회

In [9]:
# 
# 중복되는 행 삭제 참조 : https://nittaku.tistory.com/124

# 임의로 index만드는.. 
index_list = ['0','1','2']
df = pd.DataFrame(columns=['review_index','sentence','FeatureName','FeatureNameRe'])


index_count = 0

for one_review in data['review']:
    sentences = sent_tokenize(one_review)   
    
    for sentence in sentences:
        tokens = word_tokenize(sentence)
        
        for token in tokens:
            for j in range(10):
                if token in feature[str(j+12)]:
                    df = df.append(pd.DataFrame([[index_count, sentence, j+12,feature_db['FeatureNameRe'][j] ]], columns=['review_index','sentence','FeatureName','FeatureNameRe']), ignore_index=True)

                else:
                    pass
    index_count += 1
    
    
# nan값을 가진 행을 삭제. 맨 처음에 index만들기 위해 임의로 만든 행을 삭제시키는 코드
df = df.dropna(axis=0)
# review인덱스와 feature이름이 중복되는 행을 제거함. keep인자는 중복 지우고 남길 행을 의미함. last이므로 마지막행만 남긴다는 의미
df = df.drop_duplicates(['review_index','FeatureName'], keep='last')  
# 인덱스 초기화
df = df.reset_index(drop=True)


In [10]:
df

,review_index,sentence,FeatureName,FeatureNameRe
0,2,I still use my bottle of cleaning liquid and c...,19,(LCD)|(screen)
1,3,It is pretty much useless in its stock form un...,21,(flash)|(light)
2,4,currently bluetooth is not enabled but when it...,19,(LCD)|(screen)
3,5,The only downfall to using it is that it lags ...,17,(battery)|(batteries)|(power)
4,6,"The screen is nice, but the body feels a littl...",19,(LCD)|(screen)
5,7,"Other reading options are Go to Page, Back, Fi...",19,(LCD)|(screen)
6,8,It's also really easy to view the screen from ...,19,(LCD)|(screen)
7,9,Also the response on the touch screen was grea...,21,(flash)|(light)
8,9,Also the response on the touch screen was grea...,17,(battery)|(batteries)|(power)
9,9,But unlike other tablets where if you crack th...,19,(LCD)|(screen)


### class 사용... 안됨

In [11]:
# 이 셀은 쓰지 마세요


#### 줄이는 중


class runProgram :
    index_list = ['0','1','2']
    
    
    
    def __init__(self, reviews):
        self.reviews = reviews
        self.index_count = 0
        
        self.sentences = self.review2sentences(self.reviews)
        
        #######
        
        self.tokens = self.tokenizer(self.sentences)
        self.df = self.feature_labeler(self.tokens)
        self.df = self.fix_df(self.df)
        self.index_list = ['0','1','2']

#     @staticmethod
    def review2sentences(self, reviews):  
        for idx,one_review in enumerate(reviews):
            sentences = sent_tokenize(one_review)   
            return sentences


    def tokenizer(self,sentences):
        for sentence in sentences:
            tokens = word_tokenize(sentence)
            return tokens


    def feature_labeler(self,tokens):   
        df = pd.DataFrame(columns=['review_index','sentence','FeatureName','FeatureNameRe'])
        
        for token in tokens:
            for j in range(10):
                if token in feature[str(j)]:
                    df = df.append(pd.DataFrame([[idx, sentence, j+1,FeatureNameRe[str(j+1)] ]], columns=['review_index','sentence','FeatureName','FeatureNameRe']), ignore_index=True)

                else:
                    pass
        return df



    def fix_df(self,df):
        df = df.dropna(axis=0)
        df = df.drop_duplicates(['review_index','FeatureName'], keep='last')
        df = df.reset_index(drop=True)
        return df


#     def runProgram(reviews):
#         sentences = review2sentences(reviews)
#         tokens = tokenizer(sentences)
#         df = feature_labeler(tokens)
#         df = fix_df(df)

    def run() :
        for i in self.sentences :
            
        


SyntaxError: unexpected EOF while parsing (<ipython-input-11-0bbf29522ca0>, line 68)

In [ ]:
# a = runProgram(data['review'])

a.run()

In [ ]:
a.df

In [ ]:
data['review'][0]

## 3. KW 사전 구축하기

하지만 DB에 있는 사전을 쿼리문으로 가져오는 연습할 것

참조 : https://www.nltk.org/_modules/nltk/corpus/reader/comparative_sents.html

In [ ]:
nltk.download('comparative_sentences')
from nltk.corpus import comparative_sentences

comparison = comparative_sentences.keywords_readme()

In [ ]:
comparison

In [ ]:
KW = comparison[5:]

In [ ]:
KW

In [ ]:
KW_list = comparison[5:]

# 4. KW로 SS / CS 구분하기

In [ ]:
df['sentence']

In [ ]:
POS_tag_list =['JJR','RBR','JJS','RBS']
df['SS/CS'] = 0

index_count = 0


for sentence in df['sentence']:
    tokens = word_tokenize(sentence)
    tokens_tag = nltk.pos_tag(tokens)

    for token in tokens_tag:
        if token[0] in KW_list:
            df['SS/CS'][index_count] = 'CS'

        elif token[1] in POS_tag_list:
            df['SS/CS'][index_count] = 'CS'

        if df['SS/CS'][index_count] in ['CS','SS']:
            continue

        else:
            df['SS/CS'][index_count] = 'SS'
                
    index_count += 1


In [ ]:
df

In [ ]:
df[df['SS/CS'] == 'CS']

In [ ]:
df['sentence'][1118]

# 5. MPQA 사전으로 POS, NEG 라벨링

In [ ]:
db = pymysql.connect(
    host='ec2-13-209-223-88.ap-northeast-2.compute.amazonaws.com',  # DATABASE_HOST
    port=3306,
    user='nlpuser',  # DATABASE_USERNAME
    passwd='1234',  # DATABASE_PASSWORD
    db='nlp',  # DATABASE_NAME
    charset='utf8'
)

SQL = "SELECT * FROM TotalDic"
MPQA = pd.read_sql(SQL, db)
MPQA

In [ ]:
MPQA = MPQA[MPQA['DicType2'] == 'MPQA']
MPQA_list = MPQA[['Rule','Label']]
POS_list = MPQA_list[MPQA_list['Label']=='POS']
pos_list = POS_list['Rule']
pos_list = list(pos_list)
NEG_list = MPQA_list[MPQA_list['Label']=='NEG']
neg_list = NEG_list['Rule']
neg_list = list(neg_list)

In [ ]:
pos_list

In [ ]:
neg_list

In [ ]:
df['NumOfPOS'] = None
df['NumOfNEG'] = None
df['POS_token'] = None
df['NEG_token'] = None
df['Label'] = None


index_count = 0

for sentence in df['sentence']:
    tokens = word_tokenize(sentence)
    pos_count = 0
    neg_count = 0
    pos_token_list = []
    neg_token_list = []

    for token in tokens:
        if token in pos_list:
            pos_count += 1
            pos_token_list.append(token)
            

        elif token in neg_list:
            neg_count += 1
            neg_token_list.append(token)
               
        else:
            pass

    df['NumOfPOS'][index_count] = pos_count
    df['POS_token'][index_count] = pos_token_list    # 같은 token이 있어도 중복시킨다
    df['NumOfNEG'][index_count] = neg_count
    df['NEG_token'][index_count] = neg_token_list
    
    if pos_count > neg_count:
        df['Label'][index_count] = 'POS'
        
    elif pos_count < neg_count:
        df['Label'][index_count] = 'NEG'
    
    else:
        pass
    
    index_count += 1


In [ ]:
df